In [ ]:
## H2 trade analysis
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase


def H2_trade(file_path_scen,file_path_ref, run_name_scen,run_name_ref,output_folder):

    var_Fout_ref = pd.read_csv(file_path_ref + 'VAR_FOut_' + run_name_ref + '.csv', sep = ',')
    var_Fout_ref = var_Fout_ref[var_Fout_ref['1'] == 'HH2']
    var_Fout_ref = var_Fout_ref[var_Fout_ref['2'] != 'H2_STG'] 
    var_Fout_ref['2'] = var_Fout_ref['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
    var_Fout_ref['2'] = var_Fout_ref['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
    var_Fout_ref['group_cnt'] = var_Fout_ref['4'].astype(str).str.cat(var_Fout_ref['3'].astype(str), sep='_')
    var_Fout_sub_ref = var_Fout_ref[~var_Fout_ref['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]
    total_imp_ref = var_Fout_ref[var_Fout_ref['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]
    total_imp_ref = total_imp_ref.groupby('group_cnt')['8'].sum()/120
    total_prod_ref = var_Fout_sub_ref.groupby('group_cnt')['8'].sum()/120

    var_FIn_ref = pd.read_csv(file_path_ref + 'VAR_FIn_' + run_name_ref + '.csv', sep = ',')
    var_FIn_ref = var_FIn_ref[var_FIn_ref['1'] == 'HH2']
    var_FIn_ref = var_FIn_ref[var_FIn_ref['2'] != 'H2_STG']
    var_FIn_ref['2'] = var_FIn_ref['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
    var_FIn_ref['2'] = var_FIn_ref['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
    var_FIn_ref['group_cnt'] = var_FIn_ref['4'].astype(str).str.cat(var_FIn_ref['3'].astype(str), sep='_')
    var_FIn_sub_ref = var_FIn_ref[~var_FIn_ref['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]
    total_exp_ref = var_FIn_ref[var_FIn_ref['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]

    total_exp_ref = total_exp_ref.groupby('group_cnt')['8'].sum()/120
    total_cons_ref = var_FIn_sub_ref.groupby('group_cnt')['8'].sum()/120

    total_net_exp_ref = pd.merge(total_prod_ref,total_cons_ref, how='inner',on='group_cnt').reset_index(drop=False).fillna(0)
    total_net_exp_ref['net_exp'] = total_net_exp_ref['8_x'] - total_net_exp_ref['8_y']     # Example operation for negative numbers
    total_net_exp_ref['share_imp'] = (total_net_exp_ref[total_net_exp_ref['net_exp'] < 0]['net_exp']/total_net_exp_ref[total_net_exp_ref['net_exp'] < 0]['8_y'])*100
    total_net_exp_ref['share_exp'] = (total_net_exp_ref[total_net_exp_ref['net_exp'] > 0]['net_exp']/total_net_exp_ref[total_net_exp_ref['net_exp'] > 0]['8_x'])*100
    total_net_exp_ref = total_net_exp_ref.fillna(0)
    total_net_exp_ref['total_share'] = total_net_exp_ref['share_exp'] + total_net_exp_ref['share_imp']

    def drop_last_substring(s):
        substrings = s.split('_')
        return '_'.join(substrings[:-1])
    
    
    total_net_exp_ref['region'] = total_net_exp_ref['group_cnt'].apply(drop_last_substring)
    total_net_exp_ref['mean_net_shr_exp'] = total_net_exp_ref.groupby('region')['total_share'].transform('mean')
    

    var_Fout_scen = pd.read_csv(file_path_scen + 'VAR_FOut_' + run_name_scen + '.csv', sep = ',')
    var_Fout_scen = var_Fout_scen[var_Fout_scen['1'] == 'HH2']
    var_Fout_scen = var_Fout_scen[var_Fout_scen['2'] != 'H2_STG'] 
    var_Fout_scen['2'] = var_Fout_scen['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
    var_Fout_scen['2'] = var_Fout_scen['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
    var_Fout_scen['group_cnt'] = var_Fout_scen['4'].astype(str).str.cat(var_Fout_scen['3'].astype(str), sep='_')
    var_Fout_sub_scen = var_Fout_scen[~var_Fout_scen['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]
    total_imp_scen = var_Fout_scen[var_Fout_scen['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]
    total_imp_scen = total_imp_scen.groupby('group_cnt')['8'].sum()/120
    total_prod_scen = var_Fout_sub_scen.groupby('group_cnt')['8'].sum()/120

    var_FIn_scen = pd.read_csv(file_path_scen + 'VAR_FIn_' + run_name_scen + '.csv', sep = ',')
    var_FIn_scen = var_FIn_scen[var_FIn_scen['1'] == 'HH2']
    var_FIn_scen = var_FIn_scen[var_FIn_scen['2'] != 'H2_STG']
    var_FIn_scen['2'] = var_FIn_scen['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
    var_FIn_scen['2'] = var_FIn_scen['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
    var_FIn_scen['group_cnt'] = var_FIn_scen['4'].astype(str).str.cat(var_FIn_scen['3'].astype(str), sep='_')
    var_FIn_sub_sen = var_FIn_scen[~var_FIn_scen['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]
    total_exp_scen = var_FIn_scen[var_FIn_scen['2'].isin(['TU_H2Pip*','TU_H2Ship*'])]

    total_exp_scen = total_exp_scen.groupby('group_cnt')['8'].sum()/120
    total_cons_scen = var_FIn_sub_sen.groupby('group_cnt')['8'].sum()/120

    total_net_exp_scen = pd.merge(total_prod_scen,total_cons_scen, how='inner',on='group_cnt').reset_index(drop=False).fillna(0)
    total_net_exp_scen['net_exp'] = total_net_exp_scen['8_x'] - total_net_exp_scen['8_y']     # Example operation for negative numbers
    total_net_exp_scen['share_imp'] = (total_net_exp_scen[total_net_exp_scen['net_exp'] < 0]['net_exp']/total_net_exp_scen[total_net_exp_scen['net_exp'] < 0]['8_y'])*100
    total_net_exp_scen['share_exp'] = (total_net_exp_scen[total_net_exp_scen['net_exp'] > 0]['net_exp']/total_net_exp_scen[total_net_exp_scen['net_exp'] > 0]['8_x'])*100
    total_net_exp_scen = total_net_exp_scen.fillna(0)
    total_net_exp_scen['total_share'] = total_net_exp_scen['share_exp'] + total_net_exp_scen['share_imp']

    def drop_last_substring(s):
        substrings = s.split('_')
        return '_'.join(substrings[:-1])
    
    
    total_net_exp_scen['region'] = total_net_exp_scen['group_cnt'].apply(drop_last_substring)
    total_net_exp_scen['mean_net_shr_exp'] = total_net_exp_scen.groupby('region')['total_share'].transform('mean')



    country_to_region = pd.read_excel('D:/Veda/Veda_models/kinesys_test - Copie/SubRes_Tmpl/SubRES_REZoning_Sol-Win_Trans.xlsx',sheet_name ='AVA')
    country_to_region = country_to_region.iloc[3:].set_axis(country_to_region.iloc[2], axis=1).iloc[:,2:]
    country_to_region['Country'] = ''

    for index, row in country_to_region.iterrows():
        parts = row['PSET_PN'].split('-')
        country_to_region.at[index, 'Country'] = parts[1].strip()



    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) 

    total_trade_map_ref = country_to_region.merge(total_net_exp_ref, how='outer', left_on='Region', right_on='region').reset_index(drop=True).fillna(0)
    world_trade_map_ref = world.merge(total_trade_map_ref, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True).dropna()
    world_trade_map_ref.shape

    total_trade_map_scen = country_to_region.merge(total_net_exp_scen, how='outer', left_on='Region', right_on='region').reset_index(drop=True).fillna(0)
    world_trade_map_scen = world.merge(total_trade_map_scen, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True).dropna()
    total_trade_map_scen.shape


    bwr_cmap = plt.cm.bwr
    
    fig, ax = plt.subplots(figsize=(15, 8))
    trade_map = world_trade_map_scen 
    colorbar_positions = [0.42, 0.2, 0.2, 0.02]
    world.plot(ax=ax, color='white', edgecolor='black')
    trade_map.plot(column='mean_net_shr_exp', cmap=bwr_cmap, linewidth=0.8, ax=ax, edgecolor='black', legend=False)

    plt.title('Average H2 Net Export Share of the Production')
    plt.ylim([-60, 90])
    plt.axis('off')

    min_val = trade_map['mean_net_shr_exp'].min()
    max_val = trade_map['mean_net_shr_exp'].max()
    norm = Normalize(vmin=min_val, vmax=max_val)

    ax_legend = fig.add_axes(colorbar_positions)
    colorbar = ColorbarBase(ax_legend, cmap=bwr_cmap, norm=norm, orientation='horizontal')
    colorbar.set_label('H2 Net Export (%)')

    plt.savefig(output_folder + 'H2net_export_map.pdf'
            , format ='pdf',
            bbox_inches='tight')
    return plt

        


        # var_Fout['2'] = var_Fout['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
        # var_Fout['2'] = var_Fout['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
        # var_Fout['group'] = var_Fout['4'].astype(str).str.cat(var_Fout['3'].astype(str), sep='_').str.cat(var_Fout['2'].astype(str), sep='_')

        
        # var_FIn['2'] = var_FIn['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
        # var_FIn['2'] = var_FIn['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
        # var_FIn['group'] = var_FIn['4'].astype(str).str.cat(var_FIn['3'].astype(str), sep='_').str.cat(var_FIn['2'].astype(str), sep='_')

        # H2_trade_merge = pd.merge(var_FIn,var_Fout, how='outer', on='group').reset_index(drop=True).fillna(0)
        # H2_trade_merge = H2_trade_merge[~H2_trade_merge['group'].duplicated()]
        # # return H2_trade_merge

        # H2_prod['group_cnt'] = H2_prod['4'].astype(str).str.cat(H2_prod['3'].astype(str), sep='_')
        # H2_prod['total_prod'] = H2_prod.groupby('group_cnt')['8'].transform('sum')/120

        
        # # H2_prod['8'] = pd.to_numeric(H2_prod['8'])
        # H2_prod['TU_prod'] = H2_prod.groupby('group')['8'].transform('sum')/120
        # # H2_prod = H2_prod[~H2_prod['group'].duplicated()]
        # # H2_prod = H2_prod[H2_prod['2'].str.contains('TU_H2')]
        # H2_prod = H2_prod[~H2_prod['group'].duplicated()]
        # H2_prod['share_imp'] = H2_prod['TU_prod']/H2_prod['total_prod']*100
        # H2_prod = H2_prod[['2','3','4','group_cnt','group','share_imp']]
        # # return H2_prod
        # # H2_prod['share_ships'] = H2_prod['TU_prod']/H2_prod['total_prod']*100


        
        # H2_cons['2'] = H2_cons['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
        # H2_cons['2'] = H2_cons['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
        # H2_cons['group_cnt'] = H2_cons['4'].astype(str).str.cat(H2_cons['3'].astype(str), sep='_')
        # H2_cons['total_cons'] = H2_cons.groupby('group_cnt')['8'].transform('sum')/120
        

        # # H2_cons = H2_cons[~H2_cons['group'].duplicated()]
        
        # H2_cons['TU_cons'] = H2_cons.groupby('group')['8'].transform('sum')/120
        # # H2_cons = H2_cons[H2_cons['2'].str.contains('TU_H2')]
        # H2_cons = H2_cons[~H2_cons['group'].duplicated()]
        # H2_cons['share_exp'] = H2_cons['TU_cons']/H2_cons['total_cons']*100
        # H2_cons = H2_cons[['2','3','4','group_cnt','group','share_exp']]
        

        
        
        # # return H2_trade_merge
        # H2_pip = H2_trade_merge[H2_trade_merge['2_x'] == 'TU_H2Pip*']
        # H2_pip['net_pip_exp'] = H2_pip['share_exp'] - H2_pip['share_imp']

        # H2_ships = H2_trade_merge[H2_trade_merge['2_x'] == 'TU_H2Ship*']
        # H2_ships['net_ship_exp'] = H2_ships['share_exp'] - H2_ships['share_imp']

        # # H2_ships['mean_ship_exp'] = H2_ships.groupby('4_x')['net_ship_exp'].transform('mean')
        # # H2_ships = H2_ships[~H2_ships['4_x'].duplicated()]
        # # return H2_ships
        # # return H2_ships

        # country_to_region = pd.read_excel('D:/Veda/Veda_models/kinesys_test - Copie/SubRes_Tmpl/SubRES_REZoning_Sol-Win_Trans.xlsx',sheet_name ='AVA')
        # country_to_region = country_to_region.iloc[3:].set_axis(country_to_region.iloc[2], axis=1).iloc[:,2:]
        # country_to_region['Country'] = ''

        # for index, row in country_to_region.iterrows():
        #     # Split the string in the original column based on the '-' symbol
        #     parts = row['PSET_PN'].split('-')
        #     # If there are two parts after splitting, assign the second part to the new column
        #     country_to_region.at[index, 'Country'] = parts[1].strip()

        # # country_to_region.head()

        # H2ships_map = country_to_region.merge(H2_ships, how='outer', left_on='Region', right_on='4_x').reset_index(drop=True).fillna(0)
        # H2ships_map = H2ships_map.dropna()
        # H2ships_map['mean_ship_exp'] = H2ships_map.groupby('4_x')['net_ship_exp'].transform('mean')
        

        # H2pip_map = country_to_region.merge(H2_pip, how='outer', left_on='Region', right_on='4_x').reset_index(drop=True).fillna(0)
        # H2pip_map = H2pip_map.dropna()
        # H2pip_map['mean_pip_exp'] = H2pip_map.groupby('4_x')['net_pip_exp'].transform('mean')
        

        # ## map

        # world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

        # world_H2pip_map = world.merge(H2pip_map, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True)
        # world_H2pip_map.shape


        # bwr_cmap = plt.cm.bwr
        

        # # Plot the map
        # fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        # world.plot(ax=ax, color='white', edgecolor='black')
        # world_H2pip_map.plot(column='mean_pip_exp', cmap=bwr_cmap, linewidth=0.8, ax=ax, edgecolor='black', legend=False)

        # # Add title and customize plot
        # plt.title('')
        # plt.ylim([-60, 90])
        # plt.axis('off')  # Turn off axis

        # # Create a color bar
        # # Create the colorbar
        # min_val = world_H2pip_map['mean_pip_exp'].min()
        # max_val = world_H2pip_map['mean_pip_exp'].max()
        # norm = Normalize(vmin=min_val, vmax=max_val)

        # # Create an axis for the colorbar
        # ax_legend = fig.add_axes([0.4, 0.2, 0.25, 0.02])  # [left, bottom, width, height]
        # colorbar = ColorbarBase(ax_legend, cmap=bwr_cmap, norm=norm, orientation='horizontal')
        # colorbar.set_label('Pipeline Net Export')



        # ####
        # world_H2ships_map= world.merge(H2ships_map, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True)
        # world_H2ships_map.shape


        # orange_cmap = plt.cm.Oranges

        # # Plot the map
        # fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        # world.plot(ax=ax, color='white', edgecolor='black')
        # world_H2ships_map.plot(column='mean_ship_exp', cmap=bwr_cmap, linewidth=0.8, ax=ax, edgecolor='black', legend=False)

        # # Add title and customize plot
        # plt.title('')
        # plt.ylim([-60, 90])
        # plt.axis('off')  # Turn off axis

        # # Create a color bar
        # # Create the world_H2ships_map
        # min_val = world_H2ships_map['mean_ship_exp'].min()
        # max_val = world_H2ships_map['mean_ship_exp'].max()
        # norm = Normalize(vmin=min_val, vmax=max_val)

        # # Create an axis for the colorbar
        # ax_legend = fig.add_axes([0.4, 0.2, 0.25, 0.02])  # [left, bottom, width, height]
        # colorbar = ColorbarBase(ax_legend, cmap=bwr_cmap, norm=norm, orientation='horizontal')
        # colorbar.set_label('ships net export')

        # return plt
        # H2_prod['net_ships_exp'] = H2_prod['share_exp'] - H2_prod['share_imp']



        





        # H2_prod['group_cnt'] = H2_prod['4'].astype(str).str.cat(H2_prod['3'].astype(str), sep='_')
        # H2_tot_prd = H2_prod.copy()
        # H2_tot_prd['total_cons'] = H2_tot_prd.groupby('group_cnt')['8'].transform('sum')/120
        # # H2_tot_prd = H2_tot_prd[~H2_tot_prd['group'].duplicated()]

        # H2_prod_imp =  H2_prod[H2_prod['2'] == 'TU_H2Ship*']
        # H2_prod_imp = H2_prod_imp[~H2_prod_imp['group'].duplicated()]
        # H2_tot_prd = H2_tot_prd[~H2_tot_prd['group'].duplicated()]
        # H2_prd_merge = H2_tot_prd.merge(H2_prod_imp, how='left', left_on='group_cnt', right_on='group_cnt').reset_index(drop=True).dropna()
        # H2_prd_merge['share_pip'] = (H2_prd_merge['MtH2_y']/H2_prd_merge['total_cons'])*100

        
        # H2_prod_imp =  H2_prod[H2_prod['2'] == 'TU_H2Pip*']
        # H2_prod_imp = H2_prod_imp[~H2_prod_imp['group'].duplicated()]
        # H2_prd_merge1 = H2_tot_prd.merge(H2_prod_imp, how='outer', left_on='group_cnt', right_on='group_cnt').reset_index(drop=True)
        # H2_prd_merge1['share_ships'] = (H2_prd_merge1['MtH2_y']/H2_prd_merge1['total_cons'])*100

        # H2_ships = H2_prd_merge1[['3_x','4_x','share_ships']]
        # H2_ships = H2_ships[H2_ships['3_x'] == 2050]
        # H2_ships = H2_ships[~H2_ships.duplicated()]

        # H2_pip = H2_prd_merge[['3_x','4_x','share_pip']]
        # H2_pip = H2_pip[H2_pip['3_x'] == 2050]
        # H2_pip = H2_pip[~H2_pip.duplicated()]

        # var_FIn = pd.read_csv(file_path + 'VAR_FIn_' + run_name + '.csv', sep = ',')
        # H2_cons = var_FIn[var_FIn['1'] == 'HH2']
        # H2_cons = H2_cons[H2_cons['2'] != 'H2_STG']
        # H2_cons['2'] = H2_cons['2'].str.replace(r'TU_H2Ship.*', 'TU_H2Ship*', regex=True)
        # H2_cons['2'] = H2_cons['2'].str.replace(r'TU_H2Pip.*', 'TU_H2Pip*', regex=True)
        # H2_cons['group'] = H2_cons['4'].astype(str).str.cat(H2_cons['3'].astype(str), sep='_').str.cat(H2_cons['2'].astype(str), sep='_')
        # # H2_prod['8'] = pd.to_numeric(H2_prod['8'])
        # H2_cons['MtH2'] = H2_cons.groupby('group')['8'].transform('sum')/120
        # # H2_prod = H2_prod[~H2_prod['group'].duplicated()]

        # H2_cons['group_cnt'] = H2_cons['4'].astype(str).str.cat(H2_cons['3'].astype(str), sep='_')
        # H2_tot_cons = H2_cons.copy()
        # H2_tot_cons['total_cons'] = H2_tot_cons.groupby('group_cnt')['8'].transform('sum')/120
        # H2_tot_cons = H2_tot_cons[~H2_tot_cons['group'].duplicated()]

        # H2_cons_exp =  H2_cons[H2_cons['2'] == 'TU_H2Ship*']
        # H2_cons_exp = H2_cons_exp[~H2_cons_exp['group'].duplicated()]
        # H2_cons_merge1 = H2_tot_cons.merge(H2_cons_exp, how='outer', left_on='group_cnt', right_on='group_cnt').reset_index(drop=True)
        # H2_cons_merge1['share_ships'] = (H2_cons_merge1['MtH2_y']/H2_cons_merge1['total_cons'])*100

        # H2_cons_exp =  H2_cons[H2_cons['2'] == 'TU_H2Pip*']
        # H2_cons_exp = H2_cons_exp[~H2_cons_exp['group'].duplicated()]
        # H2_cons_merge = H2_tot_cons.merge(H2_cons_exp, how='outer', left_on='group_cnt', right_on='group_cnt').reset_index(drop=True)
        # H2_cons_merge['share_pip'] = (H2_cons_merge['MtH2_y']/H2_cons_merge['total_cons'])*100

        # H2_pip_exp = H2_cons_merge[['3_x','4_x','share_pip']]
        # H2_pip_exp = H2_pip_exp[H2_pip_exp['3_x'] == 2050]
        # H2_pip_exp = H2_pip_exp[~H2_pip_exp.duplicated()]

        # H2_ships_exp = H2_cons_merge1[['3_x','4_x','share_ships']]
        # H2_ships_exp = H2_ships_exp[H2_ships_exp['3_x'] == 2050]
        # H2_ships_exp = H2_ships_exp[~H2_ships_exp.duplicated()]

        # H2_ships = H2_ships_exp.merge(H2_ships, how='outer', left_on='4_x', right_on='4_x').reset_index(drop=True).fillna(0)
        # H2_ships['net_exp'] = H2_ships['share_ships_x'] - H2_ships['share_ships_y']
        # # return H2_ships
        # H2_pip = H2_pip_exp.merge(H2_pip, how='outer', left_on='4_x', right_on='4_x').reset_index(drop=True).fillna(0)
        # H2_pip['net_exp'] =  H2_pip['share_pip_x'] - H2_pip['share_pip_y']
        # # return H2_pip
        
        # # return H2_ships_exp

        # country_to_region = pd.read_excel('D:/Veda/Veda_models/kinesys_test - Copie/SubRes_Tmpl/SubRES_REZoning_Sol-Win_Trans.xlsx',sheet_name ='AVA')
        # country_to_region = country_to_region.iloc[3:].set_axis(country_to_region.iloc[2], axis=1).iloc[:,2:]
        # country_to_region['Country'] = ''

        # for index, row in country_to_region.iterrows():
        #     # Split the string in the original column based on the '-' symbol
        #     parts = row['PSET_PN'].split('-')
        #     # If there are two parts after splitting, assign the second part to the new column
        #     country_to_region.at[index, 'Country'] = parts[1].strip()

        # # country_to_region.head()

        # H2ships_map = country_to_region.merge(H2_ships, how='left', left_on='Region', right_on='4_x').reset_index(drop=True)
        # H2ships_map = H2ships_map.dropna()

        # H2pip_map = country_to_region.merge(H2_pip, how='left', left_on='Region', right_on='4_x').reset_index(drop=True)
        # H2pip_map = H2pip_map.dropna()
        

        # ## map

        # world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

        # world_H2pip_map = world.merge(H2pip_map, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True)
        # world_H2pip_map.shape


        # bwr_cmap = plt.cm.bwr
        

        # # Plot the map
        # fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        # world.plot(ax=ax, color='white', edgecolor='black')
        # world_H2pip_map.plot(column='net_exp', cmap=bwr_cmap, linewidth=0.8, ax=ax, edgecolor='black', legend=False)

        # # Add title and customize plot
        # plt.title('')
        # plt.ylim([-60, 90])
        # plt.axis('off')  # Turn off axis

        # # Create a color bar
        # # Create the colorbar
        # min_val = world_H2pip_map['net_exp'].min()
        # max_val = world_H2pip_map['net_exp'].max()
        # norm = Normalize(vmin=min_val, vmax=max_val)

        # # Create an axis for the colorbar
        # ax_legend = fig.add_axes([0.4, 0.2, 0.25, 0.02])  # [left, bottom, width, height]
        # colorbar = ColorbarBase(ax_legend, cmap=bwr_cmap, norm=norm, orientation='horizontal')
        # colorbar.set_label('Pipeline Net Export')



        # ####
        # world_H2ships_map= world.merge(H2ships_map, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True)
        # world_H2ships_map.shape


        # orange_cmap = plt.cm.Oranges

        # # Plot the map
        # fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        # world.plot(ax=ax, color='white', edgecolor='black')
        # world_H2ships_map.plot(column='net_exp', cmap=bwr_cmap, linewidth=0.8, ax=ax, edgecolor='black', legend=False)

        # # Add title and customize plot
        # plt.title('')
        # plt.ylim([-60, 90])
        # plt.axis('off')  # Turn off axis

        # # Create a color bar
        # # Create the world_H2ships_map
        # min_val = world_H2ships_map['net_exp'].min()
        # max_val = world_H2ships_map['net_exp'].max()
        # norm = Normalize(vmin=min_val, vmax=max_val)

        # # Create an axis for the colorbar
        # ax_legend = fig.add_axes([0.4, 0.2, 0.25, 0.02])  # [left, bottom, width, height]
        # colorbar = ColorbarBase(ax_legend, cmap=bwr_cmap, norm=norm, orientation='horizontal')
        # colorbar.set_label('ships net export')

# H2_trade(file_path_scen,file_path_ref, run_name_scen,run_name_ref)
        # return H2_trade_df
       
        # values1 = np.linspace(0, 1, 20)
        # values2 = np.linspace(0, 1, 9)
        # # Create two colormaps
        # cmap1 = plt.cm.tab20
        # cmap2 = plt.cm.Set3


        # colors1 = cmap1(values1)
        # colors2 = cmap2(values2)



        # # Combine the colors from both colormaps
        # combined_colors = np.vstack((colors1, colors2))

        # # Create a new colormap using the combined colors
        # combined_cmap = ListedColormap(combined_colors)
        # # cmap = ListedColormap(color_map(np.arange(num_unique)))

        # pivot_df = H2_ships.pivot(index='3_x', columns='4_x', values='share_ships')
        # column_sums = pivot_df.sum(skipna=True)

        # # Step 2: Sort the columns based on their sums
        # sorted_columns = column_sums.sort_values(ascending=False)

        # # Step 3: Reindex the DataFrame with the sorted column order
        # sorted_df = pivot_df.reindex(sorted_columns.index, axis=1)
        
        # # sorted_df.head()
        # # Plot with the discrete color map
        # sorted_df.plot(kind='line', colormap=combined_cmap)
        # plt.legend(title='', bbox_to_anchor=(0.5, -0.62), loc = 'lower center', ncol=4)
        # plt.xlabel('Year')
        # plt.xticks(rotation=0)
        # plt.ylabel('MtH2')
        # plt.title('Share Pipeline')
        # plt.show()


        # pivot_df1 = H2_pip.pivot(index='3_x', columns='4_x', values='share_pip')
        # column_sums = pivot_df1.sum(skipna=True)

        # # Step 2: Sort the columns based on their sums
        # sorted_columns = column_sums.sort_values(ascending=False)

        # # Step 3: Reindex the DataFrame with the sorted column order
        # sorted_df = pivot_df1.reindex(sorted_columns.index, axis=1)
        # # sorted_df.head()
        # # Plot with the discrete color map
        # sorted_df.plot(kind='line', colormap=combined_cmap)
        # plt.legend(title='', bbox_to_anchor=(0.5, -0.62), loc = 'lower center', ncol=4)
        # plt.xlabel('Year')
        # plt.xticks(rotation=0)
        # plt.ylabel('MtH2')
        # plt.title('Share ships')
        # plt.show()
    # plt.tight_layout()
        # plt.savefig(output_folder + 'H2_prod_country.pdf'
        #             , format ='pdf'
        #             ,  bbox_inches='tight')

    # return H2_prd_merge